## Getting data

In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import itertools
from itertools import chain
import requests # library to handle requests



pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))

In [2]:
# Statistical Areas - Three levels: Borough, Neighborood, Stat area
ds_stat_area = pd.read_csv('http://dati.comune.bologna.it/download/file/fid/1275', sep=';')
ds_stat_area.columns = ['name', 'id', 'id2', 'sqm', 'borough_id', 'borough_name', 'neighborhood_id', 'neighborhood_name', 'type']
ds_stat_area.head()

,name,id,id2,sqm,borough_id,borough_name,neighborhood_id,neighborhood_name,type
0,AEROPORTO,4,23,"4402575,4660000000",1,BORGO PANIGALE,C,Borgo Panigale,"Poli funzionali, aree produttive e infrastrutture"
1,AGUCCHI,39,20,"1092352,0940000000",4,RENO,S,Santa Viola,Tessuto consolidato e aree della trasformazion...
2,ARCOVEGGIO,14,41,"961087,3560000000",2,NAVILE,B,Bolognina,Tessuto consolidato e aree della trasformazion...
3,BARGELLINO,3,6,"696718,8840000000",1,BORGO PANIGALE,C,Borgo Panigale,"Poli funzionali, aree produttive e infrastrutture"
4,BATTINDARNO,37,35,"936825,9900000000",4,RENO,A,Barca,Tessuto consolidato e aree della trasformazion...


In [3]:
# Altre aree statistiche
#ds_aree_statistiche1 = pd.read_csv('http://dati.comune.bologna.it/download/file/fid/5162', sep=';')
#ds_aree_statistiche1.head()
#ds_aree_statistiche1.shape

In [4]:
# Income in 2017 per Statistical Area
ds_income = pd.read_csv('http://dati.comune.bologna.it/download/file/fid/5066', sep=';')
ds_income.columns = ['stat_area_name', 'tax_payer_nr', 'total_income_for_state_taxation', 'total_income_for_coucil_taxation']
ds_income.head()


,stat_area_name,tax_payer_nr,total_income_for_state_taxation,total_income_for_coucil_taxation
0,Aeroporto,324,7306571,7130548
1,Agucchi,3203,68382532,65810791
2,Arcoveggio,7819,165245894,159182761
3,Bargellino,68,1483171,1477482
4,Battindarno,6835,150912113,145989868


In [5]:
# Families per Statistical Area
ds_fam = pd.read_csv('http://dati.comune.bologna.it/file/dataset/12_famiglie_sezioni_aree_qze_sesso_annuale_1.csv', sep='\t')
ds_fam.columns = ['year', 'census_area_id', 'stat_area_id', 'stat_area_name', 'borough_name', 'neighborood_name', 'nr_of_family_members', 'nr_of_families']
ds_fam.tail()


,year,census_area_id,stat_area_id,stat_area_name,borough_name,neighborood_name,nr_of_family_members,nr_of_families
29394,2018,8888,99,non indicata,Senza fissa dimora,Senza fissa dimora,1,449
29395,2018,8888,99,non indicata,Senza fissa dimora,Senza fissa dimora,2,12
29396,2018,8888,99,non indicata,Senza fissa dimora,Senza fissa dimora,3,2
29397,2018,8888,99,non indicata,Senza fissa dimora,Senza fissa dimora,4,1
29398,2018,8888,99,non indicata,Senza fissa dimora,Senza fissa dimora,5,2


In [6]:
# Age per Statistical Area - 2018 (31/12?)
ds_age = pd.read_csv('http://dati.comune.bologna.it/download/file/fid/5094', sep='\t')
ds_age.columns = ['year', 'census_2011_section_id', 'stat_area_id', 'stat_area_name', 'borough_name', 'neighborood_name', 'sex', 'age_different_intervals', 'age_equal_intervals', 'citizenship', 'nr_of_residents']
ds_age['sex'].replace(to_replace = 'Femmine', value = 'F', inplace = True)
ds_age['sex'].replace(to_replace = 'Maschi', value = 'M', inplace = True)
ds_age['age_different_intervals'].replace('e prima', 'and before', regex = True, inplace = True)
ds_age['age_different_intervals'].replace('e oltre', 'and after', regex = True, inplace = True)
ds_age['citizenship'].replace('Italiana', 'Italian', regex = True, inplace = True)
ds_age['citizenship'].replace('Straniera', 'Foreign', regex = True, inplace = True)
ds_age.head()

,year,census_2011_section_id,stat_area_id,stat_area_name,borough_name,neighborood_name,sex,age_different_intervals,age_equal_intervals,citizenship,nr_of_residents
0,2018,1,2,Via del Vivaio,Borgo Panigale - Reno,Borgo Panigale,F,02 and before,00-04,Italian,2
1,2018,1,2,Via del Vivaio,Borgo Panigale - Reno,Borgo Panigale,F,03-05,00-04,Italian,1
2,2018,1,2,Via del Vivaio,Borgo Panigale - Reno,Borgo Panigale,F,03-05,00-04,Foreign,1
3,2018,1,2,Via del Vivaio,Borgo Panigale - Reno,Borgo Panigale,F,03-05,05-09,Italian,2
4,2018,1,2,Via del Vivaio,Borgo Panigale - Reno,Borgo Panigale,F,06-14,05-09,Italian,3


In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
import ibm_boto3
from botocore.client import Config
import json
import ibm_boto3
from botocore.client import Config
import json

cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text

json_aree = load_string(get_file('aree_statistiche.geojson')).decode("utf-8")
json_aree1 = get_file('aree_statistiche.geojson')
type(json_aree1)
#json_aree

ibm_botocore.response.StreamingBody

In [9]:
# Coordinates per Statistical Area
# get a list of all areas
dict_area = json.loads(json_aree)
area_set=set([f['properties']['NOMEAREAST'] for f in dict_area['features']])
#to create a list of areas and associated lat long
area_list=[(area,list(chain().from_iterable(f['geometry']['coordinates']))) for f in dict_area['features'] for area in area_set if f['properties']['NOMEAREAST']==area]
# Put on df
df_coord = pd.DataFrame(area_list[0:], columns=['stat_area','coords'])    
df_coord.head()
#len(df_coord.iloc[0]['coords'])

,stat_area,coords
0,VILLAGGIO DELLA BARCA,"[[11.28887243687619, 44.493839678171874], [11...."
1,BATTINDARNO,"[[11.304354670908275, 44.502231153585114], [11..."
2,CANALE DI RENO,"[[11.301304394939708, 44.490825007457346], [11..."
3,CASERME ROSSE-MANIFATTURA,"[[11.364140469163491, 44.52279866578613], [11...."
4,CNR,"[[11.34781923054498, 44.524232532194105], [11...."


In [10]:
# Centroids for each Stat Area
column_names = ["stat_area", "ctr_lat", "ctr_lng", "sw_lat", "sw_lng", "ne_lat", "ne_lng", "coords"]
df_centroids = pd.DataFrame(columns=column_names)
for index, row in df_coord.iterrows():
    lng = 0.00
    lat = 0.00
    sw_lat = 90.00
    sw_lng = 90.00
    ne_lat = 0.00
    ne_lng = 0.00
    count = 0
    for idx, val in enumerate(row['coords']):
        lng = lng + val[0]
        lat = lat + val[1]
        if sw_lng > val[0]:
            sw_lng = val[0]
        if sw_lat > val[1]:
            sw_lat = val[1]
        if ne_lng < val[0]:
            ne_lng = val[0]
        if ne_lat < val[1]:
            ne_lat = val[1]
        count = count + 1
    df_centroids.loc[-1] = [row['stat_area'], lat/count, lng/count, sw_lat, sw_lng, ne_lat, ne_lng, row['coords']]  # adding a row
    df_centroids.index = df_centroids.index + 1  # shifting index
df_centroids.head()    

,stat_area,ctr_lat,ctr_lng,sw_lat,sw_lng,ne_lat,ne_lng,coords
89,VILLAGGIO DELLA BARCA,44.495828,11.287171,44.488884,11.276946,44.506085,11.292570,"[[11.28887243687619, 44.493839678171874], [11...."
88,BATTINDARNO,44.498330,11.295966,44.492908,11.288789,44.505590,11.306803,"[[11.304354670908275, 44.502231153585114], [11..."
87,CANALE DI RENO,44.494566,11.299240,44.488746,11.289965,44.502086,11.312821,"[[11.301304394939708, 44.490825007457346], [11..."
86,CASERME ROSSE-MANIFATTURA,44.525468,11.351259,44.519118,11.338880,44.531521,11.364883,"[[11.364140469163491, 44.52279866578613], [11...."
85,CNR,44.523097,11.340673,44.518007,11.334403,44.527150,11.349361,"[[11.34781923054498, 44.524232532194105], [11...."


In [11]:
#Wrangling
ds_income['stat_area_name'] = ds_income['stat_area_name'].str.upper()
ds_income.head()

,stat_area_name,tax_payer_nr,total_income_for_state_taxation,total_income_for_coucil_taxation
0,AEROPORTO,324,7306571,7130548
1,AGUCCHI,3203,68382532,65810791
2,ARCOVEGGIO,7819,165245894,159182761
3,BARGELLINO,68,1483171,1477482
4,BATTINDARNO,6835,150912113,145989868


In [12]:
# Final dataset
#Stat area and sqm
ds_data = ds_stat_area.loc[: ,['id', 'name', 'sqm']]

#Income
#Truncate for join
ds_data['name_trunc'] = ds_data['name'].str[:20]
#Merge
ds_data = ds_data.merge(ds_income.loc[:, ['stat_area_name', 'tax_payer_nr', 'total_income_for_state_taxation']], how = 'inner', left_on = 'name_trunc', right_on = 'stat_area_name')
ds_data = ds_data.loc[: ,['id', 'name', 'sqm', 'tax_payer_nr', 'total_income_for_state_taxation']]
ds_data.rename(columns = {'tax_payer_nr' : '2017_tax_payer_nr', 'total_income_for_state_taxation' : '2017_total_income_fst'}, inplace = True)
#Attenzione: sqm è ancora a tipo dato object
ds_data['sqm'] = ds_data['sqm'].str.replace(',', '.')
ds_data['sqm'] = ds_data['sqm'].astype('float64')
ds_data['income_x_taxpayer'] = ds_data['2017_total_income_fst'] / ds_data['2017_tax_payer_nr']
ds_data.sort_values(by = 'income_x_taxpayer', ascending = False, inplace = True)
print(ds_data.dtypes)
ds_data.head()

id                         int64
name                      object
sqm                      float64
2017_tax_payer_nr          int64
2017_total_income_fst      int64
income_x_taxpayer        float64
dtype: object


,id,name,sqm,2017_tax_payer_nr,2017_total_income_fst,income_x_taxpayer
79,75,VIA DEL GENIO,3.997720e+05,12,801065,66755.416667
29,55,GIARDINI MARGHERITA,2.624597e+05,23,1381740,60075.652174
65,76,SAN LUCA,7.326767e+06,428,25041473,58508.114486
27,53,GALVANI-1,2.154302e+05,1301,72441028,55681.036126
53,52,PADERNO,2.290304e+07,1096,55334976,50488.116788


In [13]:
#Families
ds_fam_2018 = ds_fam[ds_fam.year.eq(2018)].copy()

ds_fam_2018['nr_of_family_members'].replace('Oltre 6', '7', regex = True, inplace = True)
ds_fam_2018['nr_of_family_members'] = ds_fam_2018['nr_of_family_members'].astype(int)
ds_fam_2018['total_members'] = ds_fam_2018['nr_of_family_members'] * ds_fam_2018['nr_of_families']

ds_fam_2018.tail()

grouped_df = ds_fam_2018.groupby(['stat_area_name'], as_index = False).agg({'total_members' : [sum]})

grouped_df['stat_area_name'] = grouped_df['stat_area_name'].str.upper()
grouped_df.columns = ['stat_area_name', 'total_members']
#grouped_df.head(100)
ds_data_members = ds_data.merge(grouped_df, how = 'inner', left_on = 'name', right_on = 'stat_area_name')
ds_data_members.drop(['stat_area_name'], axis=1, inplace = True)
ds_data_members['sqkm'] = ds_data_members['sqm'] / 1000000
ds_data_members['density'] = ds_data_members['total_members'] / ds_data_members['sqkm'] 
ds_data_members.sort_values(by = 'density', ascending = False, inplace = True)
ds_data_members.head(100)
#ds_data_members['total_members'].sum() / ds_data_members['sqkm'].sum()

,id,name,sqm,2017_tax_payer_nr,2017_total_income_fst,income_x_taxpayer,total_members,sqkm,density
57,83,BITONE,6.055156e+05,8219,179486006,21837.937219,10874,0.605516,17958.249550
31,72,XXI APRILE,8.249317e+05,9657,251292212,26021.767837,12898,0.824932,15635.233271
32,58,DAGNINI,6.245708e+05,7174,185881414,25910.428492,9534,0.624571,15264.883873
17,29,MARCONI-2,8.803065e+05,9237,282351058,30567.398289,12273,0.880306,13941.735177
78,49,VIA MONDO,5.787198e+05,5917,119801127,20246.937130,8049,0.578720,13908.284476
46,34,VELODROMO,4.300157e+05,4528,105125839,23216.837235,5949,0.430016,13834.377930
79,17,PIAZZA DELL'UNITA',7.771479e+05,7338,147074671,20042.882393,10537,0.777148,13558.551132
55,82,PONTEVECCHIO,4.046838e+05,3941,86911435,22053.142603,5378,0.404684,13289.388930
30,67,MENGOLI,7.802243e+05,7544,196729728,26077.641569,10070,0.780224,12906.545596
13,30,MARCONI-1,1.777344e+05,1745,64594450,37016.876791,2285,0.177734,12856.260771


In [14]:
#Age intervals
ds_age.head()
ds_age.age_equal_intervals.value_counts()

ds_age_wrk = ds_age.copy()

ds_age_wrk['age_interval_lgr'] = ''
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '00-04', '00-14', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '05-09', '00-14', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '10-14', '00-14', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '15-19', '15-29', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '20-24', '15-29', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '25-29', '15-29', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '30-34', '30-44', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '35-39', '30-44', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '40-44', '30-44', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '45-49', '45-64', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '50-54', '45-64', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '55-59', '45-64', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '60-64', '45-64', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '65-69', '65-74', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '70-74', '65-74', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '75-79', '75 and more', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '80-84', '75 and more', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '85-89', '75 and more', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '90-94', '75 and more', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '95 e oltre', '75 and more', ds_age_wrk['age_interval_lgr'])
ds_age_wrk['age_interval_lgr'] = np.where(ds_age_wrk['age_equal_intervals'] == '', '75 and more', ds_age_wrk['age_interval_lgr'])

ds_age_wrk.describe()

ds_age_wrk.isnull().sum().sum()

#Group and sum
ds_age_wrk_g = ds_age_wrk.groupby(['stat_area_name', 'age_interval_lgr'], as_index = False).agg({'nr_of_residents' : [sum]})
ds_age_wrk_g.columns = ['stat_area_name', 'age_interval_lgr', 'nr_of_residents']
ds_age_wrk_g['stat_area_name'] = ds_age_wrk_g['stat_area_name'].str.upper()

#duplicateRowsDF = ds_age_wrk_g[ds_age_wrk_g.duplicated()]
#duplicateRowsDF.head()

ds_age_wrk_g_p = ds_age_wrk_g.pivot(index = 'stat_area_name', columns = 'age_interval_lgr', values='nr_of_residents')
ds_age_wrk_g_p.reset_index(inplace = True)

print(ds_age_wrk_g_p.isnull().sum().sum())

#NaN to zero
ds_age_wrk_g_p.fillna(0, inplace = True)

#ds_age_wrk_g_p.shape
print(ds_age_wrk_g_p.isnull().sum().sum())
print(ds_age_wrk_g_p.shape)
ds_age_wrk_g_p.head()




3
0
(91, 7)


age_interval_lgr,stat_area_name,00-14,15-29,30-44,45-64,65-74,75 and more
0,AEROPORTO,84.0,53.0,135.0,156.0,35.0,17.0
1,AGUCCHI,592.0,589.0,1032.0,1236.0,407.0,561.0
2,ARCOVEGGIO,1250.0,1393.0,2546.0,3030.0,964.0,1392.0
3,BARGELLINO,12.0,11.0,15.0,37.0,3.0,5.0
4,BATTINDARNO,978.0,1112.0,1547.0,2555.0,1058.0,1617.0


In [15]:
# Merge

# New dataset
df_elab = ds_data_members.copy()

df_elab = df_elab.merge(ds_age_wrk_g_p, how = 'inner', left_on = 'name', right_on = 'stat_area_name')
df_elab.drop(['stat_area_name'], axis=1, inplace = True)

print(df_elab.shape) 

df_elab.head()

(90, 15)


,id,name,sqm,2017_tax_payer_nr,2017_total_income_fst,income_x_taxpayer,total_members,sqkm,density,00-14,15-29,30-44,45-64,65-74,75 and more
0,83,BITONE,605515.586,8219,179486006,21837.937219,10874,0.605516,17958.249550,1306.0,1299.0,2465.0,2966.0,1066.0,1758.0
1,72,XXI APRILE,824931.728,9657,251292212,26021.767837,12898,0.824932,15635.233271,1497.0,1669.0,2957.0,3601.0,1249.0,1933.0
2,58,DAGNINI,624570.752,7174,185881414,25910.428492,9534,0.624571,15264.883873,1150.0,1164.0,2081.0,2667.0,923.0,1558.0
3,29,MARCONI-2,880306.493,9237,282351058,30567.398289,12273,0.880306,13941.735177,1251.0,1634.0,3027.0,3390.0,1276.0,1706.0
4,49,VIA MONDO,578719.828,5917,119801127,20246.937130,8049,0.578720,13908.284476,943.0,1102.0,1932.0,2093.0,801.0,1240.0


## Venues

In [16]:
# @hidden cell
CLIENT_ID = 'BGVCIMLGC1P244NQR3O5WEAJVL4RLLMHPBRFJUNCQDEJWR33'
CLIENT_SECRET = 'KOUQ1X2H4DU4P1US0WLSXNDLD0DA4IS4ZWBTAETEHVNZ1G0K'
VERSION = '20180604'
LIMIT = 100

In [17]:
# Function for in-polygon selection
def dotIntersect(segment, dot, slopeDot):
    ret = []
    if float(segment[1][0]) - float(segment[0][0]) != 0.00:
        m = (segment[1][1] - segment[0][1]) / (segment[1][0] - segment[0][0])
        #print (m)
        if (m != slopeDot):
            b = segment[0][1] - m * segment[0][0]
            #print (b)
            b1 = dot[1] - slopeDot * dot[0]
            #print (b1)
            x = (b - b1) / (slopeDot - m)
            #print (x)
            y = m * (b - b1) / (slopeDot - m) + b 
            ret = [x, y]
    return ret 

def sideList(polygon):
    ret = []
    for idx, val in enumerate(polygon):
        if idx < len(polygon) - 1:
            ret.append([polygon[idx], polygon[idx + 1]])
    return ret

def inPolygon(polygon, dot):
    sides = sideList(polygon)
    count = 0
    for idx, val in enumerate(sides):
        intersect = dotIntersect(val, dot, 0.5)
        if len(intersect) > 0:
            #Greater (is on the left)
            if dot[0] < intersect[0]:
                #in side
                x_max = max([val[0][0], val[1][0]])
                x_min = min([val[0][0], val[1][0]])
                if (dot[0] > x_min and dot[0] < x_max):
                             count = count + 1
    if count % 2 == 0:
        ret = 0
    else:
        ret = -1
    
    return ret

In [18]:
# List for all areas
def fill_venue_list(results_n, df_row):
    venues_list=[]
    for index, row in results_n.iterrows():
        venues_list.append([df_row['stat_area'],
                            row['id'],
                            row['name'],
                            row['location.lng'],
                            row['location.lat'],
                            df_row['coords']])

    return(venues_list)

In [19]:
# Popolamento da tutte le aree
def venue_per_request(areas, categories):
    venue_all = []
    for index, row in areas.iterrows():
        if index == 0 or index != 0:  #For debug purposes
            url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&sw={},{}&ne={},{}&intent=browse&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, row['sw_lat'], row['sw_lng'], row['ne_lat'], row['ne_lng'], categories)
            results = requests.get(url).json()
            results_n = json_normalize(results['response']['venues'])
            venues_list = fill_venue_list(results_n, row)
            venue_all.extend(venues_list)
    df_venues = pd.DataFrame(item for item in venue_all)
    df_venues.columns = ['stat_area', 'venue_id', 'venue_name', 'venue_lng', 'venue_lat', 'area_polygon']
    return df_venues
 

In [20]:
# Add column of venues count
def add_to_df(df_target, df_coord, categories, column_name):
    df_venues = venue_per_request(df_coord, categories)
    df_venues['in'] = np.nan
    list_pol = []
    for index, row in df_venues.iterrows():
        list_pol.append(inPolygon(row['area_polygon'], [row['venue_lng'], row['venue_lat']]))
    df_venues['in'] = list_pol   
    df_venues_in = df_venues[df_venues['in'] == -1]
    df_venues_in = pd.DataFrame(df_venues_in.groupby(['stat_area']).size()).reset_index()
    df_venues_in.columns = ['stat_area', column_name]
    df_ret = df_target.merge(df_venues_in, how = 'left', left_on = 'name', right_on = 'stat_area')
    df_ret.drop(['stat_area'], axis=1, inplace = True)
    
    return df_ret 
    

In [21]:
df_elab_col = df_elab.copy()
df_elab_col = add_to_df(df_elab_col, df_centroids, '4bf58dd8d48988d1d3941735', 'veg_restaurant_nr')
df_elab_col = add_to_df(df_elab_col, df_centroids, '4bf58dd8d48988d124941735', 'office_nr')
df_elab_col = add_to_df(df_elab_col, df_centroids, '4bf58dd8d48988d13b941735', 'school_nr')
df_elab_col = add_to_df(df_elab_col, df_centroids, '4d4b7105d754a06372d81259', 'university_nr')
df_elab_col = add_to_df(df_elab_col, df_centroids, '4bf58dd8d48988d181941735', 'museum_nr')
df_elab_col.fillna(0, inplace = True)
df_elab_col.head()

,id,name,sqm,2017_tax_payer_nr,2017_total_income_fst,income_x_taxpayer,total_members,sqkm,density,00-14,15-29,30-44,45-64,65-74,75 and more,veg_restaurant_nr,office_nr,school_nr,university_nr,museum_nr
0,83,BITONE,605515.586,8219,179486006,21837.937219,10874,0.605516,17958.249550,1306.0,1299.0,2465.0,2966.0,1066.0,1758.0,1.0,10.0,5.0,3.0,0.0
1,72,XXI APRILE,824931.728,9657,251292212,26021.767837,12898,0.824932,15635.233271,1497.0,1669.0,2957.0,3601.0,1249.0,1933.0,0.0,26.0,4.0,8.0,0.0
2,58,DAGNINI,624570.752,7174,185881414,25910.428492,9534,0.624571,15264.883873,1150.0,1164.0,2081.0,2667.0,923.0,1558.0,0.0,15.0,8.0,7.0,0.0
3,29,MARCONI-2,880306.493,9237,282351058,30567.398289,12273,0.880306,13941.735177,1251.0,1634.0,3027.0,3390.0,1276.0,1706.0,2.0,15.0,18.0,20.0,3.0
4,49,VIA MONDO,578719.828,5917,119801127,20246.937130,8049,0.578720,13908.284476,943.0,1102.0,1932.0,2093.0,801.0,1240.0,0.0,7.0,4.0,5.0,0.0


In [22]:
# Keep relevant columns
df_elab_rel = df_elab_col[['name', 'income_x_taxpayer', 'density', '15-29', '30-44', '45-64', '65-74', 'veg_restaurant_nr', 'office_nr', 'school_nr', 'university_nr', 'museum_nr']].copy()
df_elab_rel['total'] = df_elab_rel[['15-29', '30-44', '45-64', '65-74']].sum(axis=1)
df_elab_rel['15-29_perc'] = df_elab_rel['15-29'] / df_elab_rel['total'] * 100
df_elab_rel['30-44_perc'] = df_elab_rel['30-44'] / df_elab_rel['total'] * 100
df_elab_rel['45-64_perc'] = df_elab_rel['45-64'] / df_elab_rel['total'] * 100
df_elab_rel['65-74_perc'] = df_elab_rel['65-74'] / df_elab_rel['total'] * 100
df_elab_rel = df_elab_rel[['name', 'income_x_taxpayer', 'density', '15-29_perc', '30-44_perc', '45-64_perc', '65-74_perc', 'veg_restaurant_nr', 'office_nr', 'school_nr', 'university_nr', 'museum_nr']].copy()

df_elab_rel.head()

,name,income_x_taxpayer,density,15-29_perc,30-44_perc,45-64_perc,65-74_perc,veg_restaurant_nr,office_nr,school_nr,university_nr,museum_nr
0,BITONE,21837.937219,17958.249550,16.662391,31.618779,38.045151,13.673679,1.0,10.0,5.0,3.0,0.0
1,XXI APRILE,26021.767837,15635.233271,17.612917,31.205150,38.001266,13.180667,0.0,26.0,4.0,8.0,0.0
2,DAGNINI,25910.428492,15264.883873,17.029993,30.446233,39.019751,13.504023,0.0,15.0,8.0,7.0,0.0
3,MARCONI-2,30567.398289,13941.735177,17.519031,32.454165,36.346092,13.680712,2.0,15.0,18.0,20.0,3.0
4,VIA MONDO,20246.937130,13908.284476,18.589744,32.591093,35.307018,13.512146,0.0,7.0,4.0,5.0,0.0


In [23]:
# Cluster
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

from sklearn.preprocessing import StandardScaler
X = df_elab_rel.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)

kclusters = 4

df_elab_col_clustering = df_elab_rel.drop(['name'], 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_elab_col_clustering)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Clus_dataSet)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

davies_bouldin_score(X, kmeans.labels_)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


1.5537565755952385

In [24]:
# Add cluster column
df_elab_rel.insert(2, 'cluster', kmeans.labels_.astype('int32'))

df_elab_rel.head()

,name,income_x_taxpayer,cluster,density,15-29_perc,30-44_perc,45-64_perc,65-74_perc,veg_restaurant_nr,office_nr,school_nr,university_nr,museum_nr
0,BITONE,21837.937219,1,17958.249550,16.662391,31.618779,38.045151,13.673679,1.0,10.0,5.0,3.0,0.0
1,XXI APRILE,26021.767837,1,15635.233271,17.612917,31.205150,38.001266,13.180667,0.0,26.0,4.0,8.0,0.0
2,DAGNINI,25910.428492,1,15264.883873,17.029993,30.446233,39.019751,13.504023,0.0,15.0,8.0,7.0,0.0
3,MARCONI-2,30567.398289,2,13941.735177,17.519031,32.454165,36.346092,13.680712,2.0,15.0,18.0,20.0,3.0
4,VIA MONDO,20246.937130,1,13908.284476,18.589744,32.591093,35.307018,13.512146,0.0,7.0,4.0,5.0,0.0


In [25]:
# Cluster description
df_cluster = df_elab_rel.groupby(['cluster'], as_index=False).mean().copy().head()
df_cluster.head()

,cluster,income_x_taxpayer,density,15-29_perc,30-44_perc,45-64_perc,65-74_perc,veg_restaurant_nr,office_nr,school_nr,university_nr,museum_nr
0,0,27849.134977,2986.171900,18.878485,25.171326,41.345785,14.604404,0.169811,7.113208,1.811321,2.169811,0.377358
1,1,25356.183249,10621.233732,17.269276,31.565288,38.098967,13.066468,0.653846,15.461538,3.923077,5.769231,0.846154
2,2,36936.794307,11403.793382,18.268855,29.993880,37.682399,14.054867,4.500000,17.000000,12.000000,19.333333,8.666667
3,3,20071.165994,373.997183,34.120594,31.774981,22.989069,11.115355,0.800000,9.600000,2.000000,8.800000,0.200000


In [26]:
# Sort
df_cluster.sort_values(by='income_x_taxpayer', inplace=True)
df_cluster.head()

,cluster,income_x_taxpayer,density,15-29_perc,30-44_perc,45-64_perc,65-74_perc,veg_restaurant_nr,office_nr,school_nr,university_nr,museum_nr
3,3,20071.165994,373.997183,34.120594,31.774981,22.989069,11.115355,0.800000,9.600000,2.000000,8.800000,0.200000
1,1,25356.183249,10621.233732,17.269276,31.565288,38.098967,13.066468,0.653846,15.461538,3.923077,5.769231,0.846154
0,0,27849.134977,2986.171900,18.878485,25.171326,41.345785,14.604404,0.169811,7.113208,1.811321,2.169811,0.377358
2,2,36936.794307,11403.793382,18.268855,29.993880,37.682399,14.054867,4.500000,17.000000,12.000000,19.333333,8.666667


In [27]:
# Bins
df_cluster_bin = df_cluster.copy()

for col in df_cluster.columns:
    if col != 'cluster':
        df_cluster_bin[col + '_bin'] = pd.cut(x=df_cluster[col], bins=3, labels=['Low', 'Medium', 'High'])
df_cluster_bin.drop(['income_x_taxpayer', 'density', '15-29_perc', '30-44_perc', '45-64_perc', '65-74_perc', 'veg_restaurant_nr', 'office_nr', 'school_nr', 'university_nr', 'museum_nr'], axis=1, inplace=True)    
df_cluster_bin.sort_values(by='cluster', inplace=True)
df_cluster_bin.head()    

,cluster,income_x_taxpayer_bin,density_bin,15-29_perc_bin,30-44_perc_bin,45-64_perc_bin,65-74_perc_bin,veg_restaurant_nr_bin,office_nr_bin,school_nr_bin,university_nr_bin,museum_nr_bin
0,0,Medium,Low,Low,Low,High,High,Low,Low,Low,Low,Low
1,1,Low,High,Low,High,High,Medium,Low,High,Low,Low,Low
2,2,High,High,Low,High,High,High,High,High,High,High,High
3,3,Low,Low,High,High,Low,Low,Low,Low,Low,Medium,Low


In [28]:
# Cluster selection
df_elab_rel_filter_sorted = df_elab_rel.loc[df_elab_rel['cluster'] == 2].copy()
df_elab_rel_filter_sorted.sort_values(by='veg_restaurant_nr', inplace=True)
df_elab_rel_filter_sorted

,name,income_x_taxpayer,cluster,density,15-29_perc,30-44_perc,45-64_perc,65-74_perc,veg_restaurant_nr,office_nr,school_nr,university_nr,museum_nr
3,MARCONI-2,30567.398289,2,13941.735177,17.519031,32.454165,36.346092,13.680712,2.0,15.0,18.0,20.0,3.0
31,GALVANI-1,55681.036126,2,8471.420230,19.060967,25.227751,41.485634,14.225648,3.0,17.0,5.0,17.0,12.0
11,GALVANI-2,37403.042157,2,12416.741189,18.108170,28.046554,39.262894,14.582382,4.0,18.0,13.0,26.0,14.0
15,IRNERIO-1,36250.047223,2,11546.030765,18.739235,29.417844,36.823975,15.018946,4.0,20.0,9.0,18.0,5.0
10,MALPIGHI-2,29785.946051,2,12600.483171,15.965001,31.904641,38.638093,13.492265,6.0,15.0,15.0,9.0,6.0
26,IRNERIO-2,31933.295992,2,9446.349759,20.220723,32.912324,33.537707,13.329246,8.0,17.0,12.0,26.0,12.0


## Maps

In [29]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [32]:
# Cluster map

bologna_map = folium.Map(location=[44.49381, 11.33875], zoom_start=12)
bologna_map.choropleth(
    geo_data=json_aree,
    data=df_elab_rel,
    columns=['name', 'cluster'],
    key_on='feature.properties.NOMEAREAST',
    fill_color='RdYlBu', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Cluster'
)
bologna_map